In [1]:
import pandas as pd
import requests
import json
import numpy as np

In [2]:
df = pd.read_csv("resources/netflix_movies.csv")
df.head()

,Title Name,Language,Released,Rating,IMDb
0,#Alive,Korean,2020,TV-MA,6.2/10
1,#AnneFrank - Parallel Stories,English,2019,TV-14,NaN
2,#FriendButMarried,Indonesian,2018,TV-G,7.1/10
3,#FriendButMarried 2,Indonesian,2020,TV-G,NaN
4,#realityhigh,English,2017,TV-14,5.2/10


In [3]:

query_url = "http://www.omdbapi.com/?apikey=7c744b18&"
query_url

'http://www.omdbapi.com/?apikey=7c744b18&'

In [5]:
data = list()
for x in df['Title Name'].tolist():
    try:
        data.append(requests.get(query_url + f"t={x}&type=movie&plot=full").json())
    except:
        data.append({"Response":"False","Error":"Movie not found!"})
data[:4]


[{'Response': 'False', 'Error': 'Incorrect IMDb ID.'},
 {'Response': 'False', 'Error': 'Incorrect IMDb ID.'},
 {'Response': 'False', 'Error': 'Incorrect IMDb ID.'},
 {'Response': 'False', 'Error': 'Incorrect IMDb ID.'}]

In [7]:
class OmdbObj():
    def __init__(self):
        self.title = np.nan
        self.year = np.nan
        self.rated = np.nan
        self.released = np.nan
        self.runtime = np.nan
        self.genre = np.nan
        self.director = np.nan
        self.writer = np.nan
        self.actors = np.nan
        self.plot = np.nan
        self.language = np.nan
        self.country = np.nan
        self.awards = np.nan
        self.poster = np.nan
        self.ratings = np.nan
        self.metascore = np.nan
        self.imdbrating = np.nan
        self.imdbvotes = np.nan
        self.imdbid = np.nan
        self.type = np.nan
        self.dvd = np.nan
        self.boxoffice = np.nan
        self.production = np.nan
        self.website = np.nan
        self.response = np.nan
    
    def _get_dict(self):
        return {x: getattr(self, x) for x in dir(self) if "_" not in x}

In [8]:
def get_dict_item(val: dict):
    temp_obj = OmdbObj()
    for k,v in val.items():
        try:
            if type(v) is not dict and val['Response'] == "True" and val['Type'] == 'movie' and not v == 'N/A':
                setattr(temp_obj, k.lower(), v)
        except KeyError:
            continue
    return temp_obj._get_dict()

In [9]:
data_list_as_dict = [get_dict_item(x) for x in data]
for x in data_list_as_dict:
    if not len(x.keys()) == 25:
        print(x)
        print(x.keys)

In [10]:
omdb_df = pd.DataFrame()
for x in data_list_as_dict:
    omdb_df = omdb_df.append(x, ignore_index=True)
omdb_df.to_csv('resources/omdb_data.csv', index=False)
omdb_df.head()

,actors,awards,boxoffice,country,director,dvd,genre,imdbid,imdbrating,imdbvotes,...,rated,ratings,released,response,runtime,title,type,website,writer,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
final_df = pd.concat([df, omdb_df], axis=1, join='inner')
final_df

,Title Name,Language,Released,Rating,IMDb,actors,awards,boxoffice,country,director,...,rated,ratings,released,response,runtime,title,type,website,writer,year
0,#Alive,Korean,2020,TV-MA,6.2/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#AnneFrank - Parallel Stories,English,2019,TV-14,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#FriendButMarried,Indonesian,2018,TV-G,7.1/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#FriendButMarried 2,Indonesian,2020,TV-G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#realityhigh,English,2017,TV-14,5.2/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4075,Zozo,Arabic,2005,TV-MA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4076,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,English,2019,TV-MA,7.4/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4077,'76,English,2016,TV-PG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4078,'83,Hindi,2021,TV-14,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
final_df.to_csv('resources/netflix_movies/final_data.csv', index=False)